## Dowload and prep the data

Use the Rock-Paper scissors dataset


In [4]:
import zipfile


#extract the archive 
local_zip = './rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./rps-train')
zip_ref.close()

local_zip = './rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./rps-test')

In [5]:
#Assign directory names 

import os 

base_dir = './rps-train/rps'

rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])


['rock04-008.png', 'rock07-k03-095.png', 'rock01-067.png', 'rock01-079.png', 'rock04-041.png', 'rock06ck02-043.png', 'rock02-096.png', 'rock07-k03-055.png', 'rock04-070.png', 'rock02-111.png']
['paper04-056.png', 'paper04-116.png', 'paper06-061.png', 'paper06-065.png', 'paper01-108.png', 'paper07-104.png', 'paper05-005.png', 'paper05-082.png', 'paper03-075.png', 'paper03-097.png']
['testscissors02-105.png', 'testscissors03-017.png', 'testscissors03-045.png', 'scissors01-070.png', 'testscissors03-093.png', 'testscissors03-010.png', 'testscissors03-036.png', 'testscissors02-081.png', 'scissors01-061.png', 'testscissors01-115.png']


## Build the model 

For the CNN, you will use 4 convolution layers with 64, 64, 128, 128 filters 

The output layer will be 3 neuron dense Softmax (scales the output probabilites so that they all add up to 1)

The order of the output will be `paper`-`rock`-`scissors`

In [12]:
import tensorflow as tf
from tensorflow.keras import layers 

model = tf.keras.models.Sequential([
        #Input shape is the desired size of the image with 3 bytes 
        #First convolution
        layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D(2, 2),
        #Second convolution
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        #Third convolution 
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        #Fourth convolution 
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        #Flatten 
        layers.Flatten(),
        #Dropout layer
        layers.Dropout(0.5),
        #512 neuron hidden layer 
        layers.Dense(512, activation='relu'),
        #Output layer 
        layers.Dense(3, activation='softmax')

])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 17, 17, 128)     

Next compile the model but change the `loss` function to `categorical_crossentropy` instead of `binary_crossentropy`

In [14]:
#Set the training parameters

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
#Prep the image data generators 

from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "./rps-train/rps"
training_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

VALIDATION_DIR = './rps-test/rps-test-set'
validation_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=126
)


validaiton_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR, 
    target_size=(150,150),
    class_mode='categorical',
    batch_size=126
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [18]:
import tensorflow as tf


#Creating a callback 

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):

    #check accuracy
    accuracy_value = 0.99
    if(logs.get('val_accuracy') > accuracy_value):

      print(f'\nAccuracy is more than {accuracy_value} so cancelling training.')
      self.model.stop_training = True

callbacks = myCallback()

In [19]:
#Train the model 

history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=14, 
    validation_data = validaiton_generator,
    verbose = 1,
    validation_steps = 3,
    callbacks=[callbacks]
)

Epoch 1/20
14/14 [==============================] - 130s 9s/step - loss: 0.0574 - accuracy: 0.9790 - val_loss: 0.1040 - val_accuracy: 0.9247
Epoch 2/20
14/14 [==============================] - 129s 9s/step - loss: 0.1180 - accuracy: 0.9660 - val_loss: 0.0609 - val_accuracy: 0.9785
Epoch 3/20
14/14 [==============================] - 129s 9s/step - loss: 0.0472 - accuracy: 0.9841 - val_loss: 0.0726 - val_accuracy: 0.9651
Epoch 4/20
14/14 [==============================] - 129s 9s/step - loss: 0.2331 - accuracy: 0.9308 - val_loss: 0.0818 - val_accuracy: 0.9785
Epoch 5/20
14/14 [==============================] - 129s 9s/step - loss: 0.0718 - accuracy: 0.9768 - val_loss: 0.0686 - val_accuracy: 0.9812
Epoch 6/20
14/14 [==============================] - ETA: 0s - loss: 0.0483 - accuracy: 0.9853
Accuracy is more than 0.99 so cancelling training.
14/14 [==============================] - 129s 9s/step - loss: 0.0483 - accuracy: 0.9853 - val_loss: 0.0116 - val_accuracy: 1.0000
